<a href="https://colab.research.google.com/github/MorningStarTM/dog-breed-classification/blob/main/dogs_breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!unzip "/content/drive/MyDrive/DataSet/rar/archive - 2023-02-15T073054.348.zip" -d "/content/drive/MyDrive/DataSet/dogs_breeds/"

In [2]:
#import libraries
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from glob import glob
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.utils import shuffle

In [3]:
#dataset directory
path = "/content/drive/MyDrive/DataSet/dogs_breeds/Images"

In [4]:
#parameters
H, W = 224, 224
channel = 3
num_class = 120
batch_size = 64

#basic functions

In [5]:
#get directory of images
def get_labels(path):
  files = os.listdir(path)
  return files

In [6]:
#split the name and unique address
def get_splited_name(labels):
  splited_list = []
  labels_list = []
  for i in labels:
    splited = i.split('-')
    labels_list.append(splited[-1])
    splited_list.append(splited)
  
  return splited_list, labels_list

In [7]:
#create dictionary for name and unique address
def get_name_index(splited_list):
  new_list = [{item[0]: item[1]} for item in splited_list]
  return new_list

In [8]:
folder = get_labels(path)
splited_list, labels_list = get_splited_name(folder)
index_list = get_name_index(splited_list)

In [9]:
labels_list

['Chihuahua',
 'Japanese_spaniel',
 'Maltese_dog',
 'Pekinese',
 'Tzu',
 'Blenheim_spaniel',
 'papillon',
 'toy_terrier',
 'Rhodesian_ridgeback',
 'Afghan_hound',
 'basset',
 'beagle',
 'bloodhound',
 'bluetick',
 'tan_coonhound',
 'Walker_hound',
 'English_foxhound',
 'redbone',
 'borzoi',
 'Irish_wolfhound',
 'Italian_greyhound',
 'whippet',
 'Ibizan_hound',
 'Norwegian_elkhound',
 'otterhound',
 'Saluki',
 'Scottish_deerhound',
 'Weimaraner',
 'Staffordshire_bullterrier',
 'American_Staffordshire_terrier',
 'Bedlington_terrier',
 'Border_terrier',
 'Kerry_blue_terrier',
 'Irish_terrier',
 'Norfolk_terrier',
 'Norwich_terrier',
 'Yorkshire_terrier',
 'haired_fox_terrier',
 'Lakeland_terrier',
 'Sealyham_terrier',
 'Airedale',
 'cairn',
 'Australian_terrier',
 'Dandie_Dinmont',
 'Boston_bull',
 'miniature_schnauzer',
 'giant_schnauzer',
 'standard_schnauzer',
 'Scotch_terrier',
 'Tibetan_terrier',
 'silky_terrier',
 'coated_wheaten_terrier',
 'West_Highland_white_terrier',
 'Lhasa',
 

In [10]:
index_list

[{'n02085620': 'Chihuahua'},
 {'n02085782': 'Japanese_spaniel'},
 {'n02085936': 'Maltese_dog'},
 {'n02086079': 'Pekinese'},
 {'n02086240': 'Shih'},
 {'n02086646': 'Blenheim_spaniel'},
 {'n02086910': 'papillon'},
 {'n02087046': 'toy_terrier'},
 {'n02087394': 'Rhodesian_ridgeback'},
 {'n02088094': 'Afghan_hound'},
 {'n02088238': 'basset'},
 {'n02088364': 'beagle'},
 {'n02088466': 'bloodhound'},
 {'n02088632': 'bluetick'},
 {'n02089078': 'black'},
 {'n02089867': 'Walker_hound'},
 {'n02089973': 'English_foxhound'},
 {'n02090379': 'redbone'},
 {'n02090622': 'borzoi'},
 {'n02090721': 'Irish_wolfhound'},
 {'n02091032': 'Italian_greyhound'},
 {'n02091134': 'whippet'},
 {'n02091244': 'Ibizan_hound'},
 {'n02091467': 'Norwegian_elkhound'},
 {'n02091635': 'otterhound'},
 {'n02091831': 'Saluki'},
 {'n02092002': 'Scottish_deerhound'},
 {'n02092339': 'Weimaraner'},
 {'n02093256': 'Staffordshire_bullterrier'},
 {'n02093428': 'American_Staffordshire_terrier'},
 {'n02093647': 'Bedlington_terrier'},
 {'n

In [11]:
len(index_list)

120

#Data Pipeline

In [12]:
def load_data(path, split=0.2):
  images = shuffle(glob(os.path.join(path,"*","*.jpg")))
  split_size = int(len(images)*split)
  #split data
  train_data, valid_data = train_test_split(images, test_size=split_size, random_state=42)
  train_data, test_data = train_test_split(train_data, test_size=split_size, random_state=42)

  return train_data, valid_data, test_data

In [13]:
train_data, valid_data, test_data = load_data(path)

In [14]:
print(f'train data: {len(train_data)}')
print(f'test data: {len(test_data)}')
print(f'valid data: {len(valid_data)}')

train data: 12348
test data: 4116
valid data: 4116


In [15]:
train_data[0]

'/content/drive/MyDrive/DataSet/dogs_breeds/Images/n02088238-basset/n02088238_7233.jpg'

In [16]:
def process_image(path):
    #decode the path
    path = path.decode()
    #read image
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    #resize the image
    image = cv2.resize(image, [W, H])
    #scale the image
    image = image / 255.0
    #change the data type of image
    image = image.astype(np.float32)

    #labeling the image
    class_name = path.split("/")[-2].split('-')[-1]
    class_idx = labels_list.index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return image, class_idx

In [17]:
def parse(path):
    image, labels = tf.numpy_function(process_image, [path], (tf.float32, tf.int32))
    labels = tf.one_hot(labels, num_class)
    image.set_shape([224, 224, 3])
    labels.set_shape(num_class)
  
    return image, labels

In [18]:
#tensorflow dataset
def tf_dataset(images, batch=64):
    dataset = tf.data.Dataset.from_tensor_slices((images))
    dataset = dataset.map(parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(64)
    return dataset

In [19]:
#Model (CNN)

In [20]:
#input layer
inputs = Input(shape=(W, H, channel))

In [21]:
#convolutional layer
conv_x = Conv2D(16, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(inputs)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)
conv_x = Conv2D(16, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)

conv_x = Conv2D(32, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = MaxPooling2D((2,2), strides=(1,1))(conv_x)

conv_x = Conv2D(64, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)
conv_x = Conv2D(64, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)

conv_x = Conv2D(128, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)
conv_x = Conv2D(128, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)

conv_x = Conv2D(256, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)
conv_x = Conv2D(256, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)

conv_x = Conv2D(512, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)
conv_x = MaxPooling2D((2,2), strides=(2,2))(conv_x)
conv_x = Conv2D(512, (3,3), activation='relu', padding='same', strides=(1,1), kernel_initializer='he_normal')(conv_x)

#flatting
flatten = Flatten()(conv_x)
conv_x = Dense(64, activation='relu')(flatten)

#adding Dense layer with number of class 
outputs = Dense(num_class, activation='softmax')(conv_x)

In [22]:
model = Model(inputs=inputs, outputs=outputs)

In [23]:
#summary of model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 16)      2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                         

In [24]:
#callbacks parameter //paths
model_path = "/content/drive/MyDrive/CNN_Models/dog_breed.h5"
csv_path = "/content/drive/MyDrive/Model CSV/dog_breed.csv"

In [25]:
#initalize callbacks
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    CSVLogger(csv_path),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=1e-7, verbose=1)
]

In [26]:
#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
#tensor dataset
train_df = tf_dataset(train_data)
test_df = tf_dataset(test_data)
valid_df = tf_dataset(valid_data) 

In [ ]:
#training
model.fit(
    train_df,
    validation_data=valid_df,
    epochs=20,
    batch_size=128,
    callbacks=callbacks
)

Epoch 1/20
 24/193 [==>...........................] - ETA: 1:59:50 - loss: 4.8660 - accuracy: 0.0091